### Danger Level Forecasting Preliminary Models

Testing out some of the classification models

#### Import Tools

In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

#### Import Data

In [26]:
avi = pd.read_csv('snowweatheModel.csv')
avi.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0_x,AVY_DANGER,AWND,SNOW,FIVE_DAY_SNOWFALL,TMAX_SWING,TMAX_SWING_FROM_AVE,WDF5,...,day_y,prevailing_wind_E,prevailing_wind_N,prevailing_wind_NE,prevailing_wind_NW,prevailing_wind_S,prevailing_wind_SE,prevailing_wind_SW,prevailing_wind_W,prevailing_wind_na
0,0,0,0,2.0,10.74,0.2,NaN,NaN,NaN,320.0,...,18,0,0,0,1,0,0,0,0,0
1,1,1,1,1.0,9.40,0.1,NaN,3.0,NaN,180.0,...,19,0,0,0,1,0,0,0,0,0
2,2,2,2,1.0,20.58,2.2,NaN,0.0,NaN,360.0,...,20,0,1,0,0,0,0,0,0,0
3,3,3,3,3.0,35.12,2.0,NaN,3.0,NaN,360.0,...,21,0,1,0,0,0,0,0,0,0
4,4,4,4,2.0,33.78,4.1,8.6,-3.0,NaN,360.0,...,22,0,1,0,0,0,0,0,0,0


#### filter data

In [27]:
avi = avi.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [28]:
avi = avi[avi['AVY_DANGER'].notnull()]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 1356
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AVY_DANGER            1254 non-null   float64
 1   AWND                  1225 non-null   float64
 2   SNOW                  1253 non-null   float64
 3   FIVE_DAY_SNOWFALL     1244 non-null   float64
 4   TMAX_SWING            1251 non-null   float64
 5   TMAX_SWING_FROM_AVE   1242 non-null   float64
 6   WDF5                  1225 non-null   float64
 7   year_x                1254 non-null   float64
 8   month_x               1254 non-null   float64
 9   day_x                 1254 non-null   float64
 10  temp_max              1254 non-null   int64  
 11  temp_min              1254 non-null   int64  
 12  water_equivalent      1254 non-null   float64
 13  snow_fall             1254 non-null   float64
 14  snow_depth_6am        1254 non-null   float64
 15  wind_speed_sum       

#### Fill Remaing NA's

In [44]:
avi = avi.fillna(0)
avi = avi[avi.AVY_DANGER != 5]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1212 entries, 5 to 1328
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   AVY_DANGER            1212 non-null   float64
 1   AWND                  1212 non-null   float64
 2   SNOW                  1212 non-null   float64
 3   FIVE_DAY_SNOWFALL     1212 non-null   float64
 4   TMAX_SWING            1212 non-null   float64
 5   TMAX_SWING_FROM_AVE   1212 non-null   float64
 6   WDF5                  1212 non-null   float64
 7   year_x                1212 non-null   float64
 8   month_x               1212 non-null   float64
 9   day_x                 1212 non-null   float64
 10  temp_max              1212 non-null   int64  
 11  temp_min              1212 non-null   int64  
 12  water_equivalent      1212 non-null   float64
 13  snow_fall             1212 non-null   float64
 14  snow_depth_6am        1212 non-null   float64
 15  wind_speed_sum       

#### Splitting Dataset

In [45]:
x = avi.iloc[:, 1:avi.shape[1]]
y = avi.iloc[:, 0]

print(x.shape)
print(y.shape)

(1212, 30)
(1212,)


In [46]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [47]:
print(y)

(unique, counts) = np.unique(y, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)


5       3.0
6       2.0
7       2.0
8       2.0
9       4.0
       ... 
1324    1.0
1325    1.0
1326    3.0
1327    3.0
1328    2.0
Name: AVY_DANGER, Length: 1212, dtype: float64
[[  1. 355.]
 [  2. 457.]
 [  3. 332.]
 [  4.  68.]]


### Descision Trees 

In [68]:
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              }


ds = DecisionTreeClassifier()
ds_cv = RandomizedSearchCV(estimator=ds, param_distributions=random_grid, n_iter=100, scoring='f1_weighted')
ds_cv.fit(x_train, y_train)
y_pred_ds = ds_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_ds))
print(cross_val_score(ds, x_train, y_train, cv=3))

0.5676567656765676
[0.55775578 0.52145215 0.58085809]


In [49]:
print(confusion_matrix(y_test, y_pred_ds))

[[56 21 10  0]
 [19 68 28  0]
 [14 33 33  6]
 [ 1  3  6  5]]


In [50]:

print(classification_report(y_test, y_pred_ds))

              precision    recall  f1-score   support

         1.0       0.62      0.64      0.63        87
         2.0       0.54      0.59      0.57       115
         3.0       0.43      0.38      0.40        86
         4.0       0.45      0.33      0.38        15

    accuracy                           0.53       303
   macro avg       0.51      0.49      0.50       303
weighted avg       0.53      0.53      0.53       303



### Random Forest

In [57]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


rf = RandomForestClassifier(class_weight = "balanced")
rf_cv = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=5, scoring='f1_weighted')
rf_cv.fit(x_train, y_train)
y_pred_rf = rf_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_rf))
print(cross_val_score(rf, x_train, y_train, cv=3))

0.6732673267326733
[0.63366337 0.62376238 0.64686469]


In [58]:
print(confusion_matrix(y_test, y_pred_rf))

[[64 14  9  0]
 [ 8 86 21  0]
 [ 5 27 49  5]
 [ 0  1  9  5]]


In [59]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

         1.0       0.83      0.74      0.78        87
         2.0       0.67      0.75      0.71       115
         3.0       0.56      0.57      0.56        86
         4.0       0.50      0.33      0.40        15

    accuracy                           0.67       303
   macro avg       0.64      0.60      0.61       303
weighted avg       0.68      0.67      0.67       303



### Extra Forest

In [60]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

et = ExtraTreesClassifier(class_weight = "balanced")
et_cv = RandomizedSearchCV(estimator=et, param_distributions=random_grid, n_iter=5, scoring='f1_weighted')
et_cv.fit(x_train, y_train)
y_pred_et = et_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_et))
print(cross_val_score(rf, x_train, y_train, cv=3))

0.6336633663366337
[0.62706271 0.62046205 0.64686469]


In [61]:
print(confusion_matrix(y_test, y_pred_et))

[[54 25  8  0]
 [11 89 15  0]
 [ 6 32 48  0]
 [ 0  1 13  1]]


In [62]:
print(classification_report(y_test, y_pred_et))

              precision    recall  f1-score   support

         1.0       0.76      0.62      0.68        87
         2.0       0.61      0.77      0.68       115
         3.0       0.57      0.56      0.56        86
         4.0       1.00      0.07      0.12        15

    accuracy                           0.63       303
   macro avg       0.73      0.50      0.51       303
weighted avg       0.66      0.63      0.62       303

